### Thông tin về GPU

In [1]:
from numba import cuda
major, minor = cuda.get_current_device().compute_capability
print(f'GPU compute capability: {major}.{minor}')

GPU compute capability: 7.5


### Khởi tạo project

In [2]:
%cd /content
!rm -rf cuda
!git clone -b main https://github.com/vietddude/cuda.git

/content
Cloning into 'cuda'...
remote: Enumerating objects: 644, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (134/134), done.
remote: Total 644 (delta 117), reused 122 (delta 52), pack-reused 452
Receiving objects: 100% (644/644), 42.53 MiB | 21.59 MiB/s, done.
Resolving deltas: 100% (173/173), done.


In [3]:
%cd /content/cuda

/content/cuda


In [4]:
!make clean
!make setup

rm -f save_train_parameters test test_cpu
rm -f *.o src/*.o src/layer/*.o src/loss/*.o src/optimizer/*.o src/layer/kernel/*.o
make network.o
make[1]: Entering directory '/content/cuda'
nvcc -arch=sm_75 --compile src/network.cc -o src/network.o -I./ -L/usr/local/cuda/lib64 -lcudart
make[1]: Leaving directory '/content/cuda'
make mnist.o
make[1]: Entering directory '/content/cuda'
nvcc -arch=sm_75 --compile src/mnist.cc -o src/mnist.o  -I./ -L/usr/local/cuda/lib64 -lcudart
make[1]: Leaving directory '/content/cuda'
make layer
make[1]: Entering directory '/content/cuda'
nvcc -arch=sm_75 --compile src/layer/ave_pooling.cc -o src/layer/ave_pooling.o -I./ -L/usr/local/cuda/lib64 -lcudart
nvcc -arch=sm_75 --compile src/layer/conv.cc -o src/layer/conv.o -I./ -L/usr/local/cuda/lib64 -lcudart
nvcc -arch=sm_75 --compile src/layer/conv_kernel.cc -o src/layer/conv_kernel.o -I./ -L/usr/local/cuda/lib64 -lcudart
nvcc -arch=sm_75 --compile src/layer/fully_connected.cc -o src/layer/fully_connected.o -I

### Build và chạy file `test_cpu`

In [5]:
!make clean_test_cpu
!make kernel_0
!make test_cpu
!./test_cpu

rm -f test_cpu
rm -f *.o 
rm -f src/layer/kernel/*.o
nvcc -arch=sm_75 --compile src/layer/kernel/kernel.cu -o src/layer/kernel/kernel.o -I./ -L/usr/local/cuda/lib64 -lcudart 
nvcc -arch=sm_75 --compile src/layer/kernel/kernel_forward_0.cu -o src/layer/kernel/kernel_forward.o -I./ -L/usr/local/cuda/lib64 -lcudart
nvcc -arch=sm_75 --compile test_cpu.cc -I./ -L/usr/local/cuda/lib64 -lcudart
nvcc -arch=sm_75 --compile src/layer/kernel/kernel.cu -o src/layer/kernel/kernel.o -I./ -L/usr/local/cuda/lib64 -lcudart 
nvcc -arch=sm_75 -o test_cpu -lm -lcuda -lrt test_cpu.o src/network.o src/mnist.o src/layer/*.o src/loss/*.o src/optimizer/*.o src/layer/kernel/*.o -I./ -L/usr/local/cuda/lib64 -lcudart
**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************
Reading: ./data/fashion/train-images-idx3-ubyte
Reading: ./data/

### Build và chạy file kernel cơ bản

In [6]:
!make kernel_0
!make clean_test
!make test
!./test

rm -f src/layer/kernel/*.o
nvcc -arch=sm_75 --compile src/layer/kernel/kernel.cu -o src/layer/kernel/kernel.o -I./ -L/usr/local/cuda/lib64 -lcudart 
nvcc -arch=sm_75 --compile src/layer/kernel/kernel_forward_0.cu -o src/layer/kernel/kernel_forward.o -I./ -L/usr/local/cuda/lib64 -lcudart
rm -f save_train_parameters test
rm -f *.o 
nvcc -arch=sm_75 --compile test.cc -I./ -L/usr/local/cuda/lib64 -lcudart
nvcc -arch=sm_75 -o test -lm -lcuda -lrt test.o src/network.o src/mnist.o src/layer/*.o src/loss/*.o src/optimizer/*.o src/layer/kernel/*.o -I./ -L/usr/local/cuda/lib64 -lcudart
**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************
Reading: ./data/fashion/train-images-idx3-ubyte
Reading: ./data/fashion/t10k-images-idx3-ubyte
Reading: ./data/fashion/train-labels-idx1-ubyte
Reading: ./data/fashion/t10k-labels-i

### Build và khởi chạy file tối ưu sử dụng unroll và nhân ma trận

In [7]:
!make kernel_1
!make clean_test
!make test
!./test

rm -f src/layer/kernel/*.o
nvcc -arch=sm_75 --compile src/layer/kernel/kernel.cu -o src/layer/kernel/kernel.o -I./ -L/usr/local/cuda/lib64 -lcudart 
nvcc -arch=sm_75 --compile src/layer/kernel/kernel_forward_1.cu -o src/layer/kernel/kernel_forward.o -I./ -L/usr/local/cuda/lib64 -lcudart
rm -f save_train_parameters test
rm -f *.o 
nvcc -arch=sm_75 --compile test.cc -I./ -L/usr/local/cuda/lib64 -lcudart
nvcc -arch=sm_75 -o test -lm -lcuda -lrt test.o src/network.o src/mnist.o src/layer/*.o src/loss/*.o src/optimizer/*.o src/layer/kernel/*.o -I./ -L/usr/local/cuda/lib64 -lcudart
**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************
Reading: ./data/fashion/train-images-idx3-ubyte
Reading: ./data/fashion/t10k-images-idx3-ubyte
Reading: ./data/fashion/train-labels-idx1-ubyte
Reading: ./data/fashion/t10k-labels-i

### Build và khởi chạy file tối ưu sử dụng tiled shared memory

In [8]:
!make kernel_2
!make clean_test
!make test
!./test

rm -f src/layer/kernel/*.o
nvcc -arch=sm_75 --compile src/layer/kernel/kernel.cu -o src/layer/kernel/kernel.o -I./ -L/usr/local/cuda/lib64 -lcudart 
nvcc -arch=sm_75 --compile src/layer/kernel/kernel_forward_2.cu -o src/layer/kernel/kernel_forward.o -I./ -L/usr/local/cuda/lib64 -lcudart
rm -f save_train_parameters test
rm -f *.o 
nvcc -arch=sm_75 --compile test.cc -I./ -L/usr/local/cuda/lib64 -lcudart
nvcc -arch=sm_75 -o test -lm -lcuda -lrt test.o src/network.o src/mnist.o src/layer/*.o src/loss/*.o src/optimizer/*.o src/layer/kernel/*.o -I./ -L/usr/local/cuda/lib64 -lcudart
**********GPU info**********
Name: Tesla T4
Compute capability: 7.5
Num SMs: 40
Max num threads per SM: 1024
Max num warps per SM: 32
GMEM: 15835660288 byte
SMEM per SM: 65536 byte
SMEM per block: 49152 byte
****************************
Reading: ./data/fashion/train-images-idx3-ubyte
Reading: ./data/fashion/t10k-images-idx3-ubyte
Reading: ./data/fashion/train-labels-idx1-ubyte
Reading: ./data/fashion/t10k-labels-i